# New Samples

RayStation plan "goalObjs"

## Clinical Goals
idx | Name | Description | Type | GoalCriteria | AcceptanceLevel | ParameterValue
-|-|-|-|-|-|-
3 | Heart | At most 15 cm$^3$ volume at 2800 cGy dose | AbsoluteVolumeAtDose | AtMost | 15 | 2800
9 | Lungs | At most 10 % volume at 2000 cGy dose | VolumeAtDose | AtMost | 0.1 | 2000
11 | SpinalCanal |  At most 2080 cGy dose at 0.35 cm$^3$ volume | DoseAtAbsoluteVolume| AtMost | 2080 | 0.35
14 | Rib | At most 1 cm$^3$ volume at 3200 cGy dose | AbsoluteVolumeAtDose | AtMost | 1 | 3200
18 | Chestwall_L | At most 30 cm$^3$ volume at 3000 cGy dose | AbsoluteVolumeAtDose | AtMost | 30 | 3000
19 | PTV | At least 95 % volume at 4800 cGy dose | VolumeAtDose | AtLeast | 0.95 | 4800
20 | PTV | At least 99 % volume at 4320 cGy dose | VolumeAtDose | AtLeast | 0.99 | 4320

### Modifications

For the DVH objectives specified in cm$^3$, I calculated percents based on total volume:

Name | Goal Volume | Total Volume | Percent Volume
-|-|-|-
Rib | 1 | 367.26 | 0.27
SpinalCanal | 0.35 | 51.93 | 0.67
Chestwall_L | 30 | 1468.53 | 2.04
Heart | 15 | 816.05 | 1.84


## Objective Terms

I am choosing not to tune the PTV MinDose parameter, since that's the value that I will end up normalizing it to in the end. Unlike the previous samples I've looked at, for this run I removed all other objective function terms.

idx | Name | Description | FunctionType | DoseValue | PercentVolume | Weight
-|-|-|-|-|-|-
0 | PTV | Min Dose 4800 cGy | MinDose | 4800 | 0 | 1
1 | PTV | Max Dose 6200 cGy | MaxDose | 6200 | 0 | 1
2 | Rib | Max DVH 3200 cGy to 0.27% volume | MaxDvh | 3200 | 0.27 | 1
3 | SpinalCanal | Max DVH 2080 cGy to 0.67% volume | MaxDvh | 2080 | 0.67 | 1
4 | Heart | Max DVH 2800 to 2% volume | MaxDvh | 2800 | 1.84 | 1
5 | Chestwall_L | Max DVH 3000 cGy to 2.04% volume | MaxDvh | 3000 | 2.04 | 1
6 | Lungs | Max DVH 2000 cGy to 10% volume | MaxDvh | 2000 | 10 | 1

In [ ]:
# Work on modifying just the dose values
# What ranges to samples from?

# I think I'll also add goals that I'll grab based directly on the objective
# (Some of the goals are in terms of the percent rather than the dose)



In [ ]:
# TODO

# get_goals(): Create new one for my specific objective values
# Choose parameter ranges
# How many samples to grab?

In [3]:
Guesing about 250 samples, about 5 hours (estimate)

In [7]:
6198

1.4705882352941178

In [13]:
6200 - (6200 - 4800)/2

5500.0

In [ ]:
# go until get 100 good plans, keep flag, keep normalized values

In [ ]:
# Check max iteration and when conversion happens (these are taking much less iterations to converge)
# Might converge faster because I have less objective terms